In [53]:
import geopandas as gpd
import pandas as pd
from pandas import DataFrame

In [55]:
path = "wrm_demographics_2022-12-05.csv"
ir_fields =['description','geometry',]
data_0 = pd.read_csv(path)
data = data_0.drop(ir_fields,axis=1)
data

,id,date,date_reported,type,birth_year,gender,race,disability,disability_type,mobility_aid,mobility_aid_type,feature_type,feature_subtype,lon,lat,gender_class,age,age_class,mobility_aid_type_class,disability_type_class
0,4,2021-05-27T22:34:00Z,2021-05-28T22:50:36Z,incident,1971,female,white,no,NaN,no response,NaN,near-miss,vehicle-turning-head-on,-123.434508,48.459019,Women,50,45-54,No response,None
1,7,2021-05-29T19:46:42Z,2021-05-29T20:01:09Z,incident,2000,no-response,no-response,no,NaN,no response,NaN,near-miss,vehicle-turning-right,-123.316472,48.433674,No Response,21,18-24,No response,None
2,8,2021-05-31T17:54:58Z,2021-05-31T18:00:16Z,amenity,1982,male,white,no,NaN,no response,NaN,sidewalk,NaN,-123.039638,49.288649,Men,39,35-44,No response,None
3,16,2021-06-04T17:31:46Z,2021-06-04T17:56:33Z,incident,1982,male,white,no,NaN,no response,NaN,near-miss,vehicle-from-behind,-123.036018,49.281230,Men,39,35-44,No response,None
4,17,2021-06-04T21:35:00Z,2021-06-04T21:41:11Z,amenity,1993,female,white,no,NaN,no response,NaN,transit,NaN,-123.193901,49.263620,Women,28,25-34,No response,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1383,1987,2022-11-27T18:00:00Z,2022-12-04T20:03:10Z,hazard-concern,1975,male,black,no,NaN,no response,NaN,safety/comfort concern,vehicles - speed makes uncomfortable,-75.700354,45.497378,Men,47,45-54,No response,None
1384,1988,2022-12-04T19:55:30Z,2022-12-04T20:05:26Z,hazard-concern,1975,female,black,no,NaN,no response,NaN,safety/comfort concern,vehicles - speed makes uncomfortable,-75.699729,45.501452,Women,47,45-54,No response,None
1385,1989,2022-12-04T19:55:30Z,2022-12-04T20:08:21Z,amenity,1986,male,white,no,NaN,no response,NaN,sidewalk,NaN,-75.696871,45.496891,Men,36,35-44,No response,None
1386,1990,2022-12-04T19:55:30Z,2022-12-04T20:11:31Z,amenity,1962,male,latino,no,NaN,no response,NaN,sidewalk,NaN,-75.691793,45.504402,Men,60,55-64,No response,None


In [56]:
for i in range(1,len(data.columns)):
    print(DataFrame(data.groupby(data.columns[i]).count()['id'].rename('count').sort_values(ascending=False)))
    print('\n')

                      count
date                       
2022-03-01T16:06:54Z     19
2022-11-02T16:36:45Z     11
2022-11-09T20:02:47Z     10
2022-02-01T19:05:13Z      8
2022-05-11T17:08:24Z      8
...                     ...
2022-02-01T16:06:00Z      1
2022-02-01T16:09:17Z      1
2022-02-01T16:49:59Z      1
2022-02-01T18:17:17Z      1
2022-04-15T17:50:00Z      1

[1084 rows x 1 columns]


                      count
date_reported              
2022-05-11T18:17:20Z      3
2021-12-08T16:20:36Z      3
2022-08-21T19:14:12Z      2
2022-01-10T03:12:37Z      2
2022-08-11T03:27:50Z      1
...                     ...
2022-02-24T17:18:05Z      1
2022-02-24T17:08:02Z      1
2022-02-24T16:56:41Z      1
2022-02-24T16:50:43Z      1
2022-12-04T20:13:56Z      1

[1382 rows x 1 columns]


                count
type                 
hazard-concern    737
amenity           475
incident          176


            count
birth_year       
1986          102
1997           51
1985           50
1971           4

#### Time (time and weekend VS week)

In [58]:
date = data['date'][0]
date

'2021-05-27T22:34:00Z'

In [68]:
data['timestamp'] = pd.Timestamp(data['date'])

TypeError: Cannot convert input [0       2021-05-27T22:34:00Z
1       2021-05-29T19:46:42Z
2       2021-05-31T17:54:58Z
3       2021-06-04T17:31:46Z
4       2021-06-04T21:35:00Z
                ...         
1383    2022-11-27T18:00:00Z
1384    2022-12-04T19:55:30Z
1385    2022-12-04T19:55:30Z
1386    2022-12-04T19:55:30Z
1387    2022-12-04T19:55:30Z
Name: date, Length: 1388, dtype: object] of type <class 'pandas.core.series.Series'> to Timestamp

In [65]:
data[['timestamp','date']]

KeyError: "['timestamp'] not in index"

In [66]:
data

,id,date,date_reported,type,birth_year,gender,race,disability,disability_type,mobility_aid,mobility_aid_type,feature_type,feature_subtype,lon,lat,gender_class,age,age_class,mobility_aid_type_class,disability_type_class
0,4,2021-05-27T22:34:00Z,2021-05-28T22:50:36Z,incident,1971,female,white,no,NaN,no response,NaN,near-miss,vehicle-turning-head-on,-123.434508,48.459019,Women,50,45-54,No response,None
1,7,2021-05-29T19:46:42Z,2021-05-29T20:01:09Z,incident,2000,no-response,no-response,no,NaN,no response,NaN,near-miss,vehicle-turning-right,-123.316472,48.433674,No Response,21,18-24,No response,None
2,8,2021-05-31T17:54:58Z,2021-05-31T18:00:16Z,amenity,1982,male,white,no,NaN,no response,NaN,sidewalk,NaN,-123.039638,49.288649,Men,39,35-44,No response,None
3,16,2021-06-04T17:31:46Z,2021-06-04T17:56:33Z,incident,1982,male,white,no,NaN,no response,NaN,near-miss,vehicle-from-behind,-123.036018,49.281230,Men,39,35-44,No response,None
4,17,2021-06-04T21:35:00Z,2021-06-04T21:41:11Z,amenity,1993,female,white,no,NaN,no response,NaN,transit,NaN,-123.193901,49.263620,Women,28,25-34,No response,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1383,1987,2022-11-27T18:00:00Z,2022-12-04T20:03:10Z,hazard-concern,1975,male,black,no,NaN,no response,NaN,safety/comfort concern,vehicles - speed makes uncomfortable,-75.700354,45.497378,Men,47,45-54,No response,None
1384,1988,2022-12-04T19:55:30Z,2022-12-04T20:05:26Z,hazard-concern,1975,female,black,no,NaN,no response,NaN,safety/comfort concern,vehicles - speed makes uncomfortable,-75.699729,45.501452,Women,47,45-54,No response,None
1385,1989,2022-12-04T19:55:30Z,2022-12-04T20:08:21Z,amenity,1986,male,white,no,NaN,no response,NaN,sidewalk,NaN,-75.696871,45.496891,Men,36,35-44,No response,None
1386,1990,2022-12-04T19:55:30Z,2022-12-04T20:11:31Z,amenity,1962,male,latino,no,NaN,no response,NaN,sidewalk,NaN,-75.691793,45.504402,Men,60,55-64,No response,None
